In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate, BatchNormalization, ReLU, LeakyReLU, Add
from tensorflow.keras.models import Model


In [3]:
# Define image dimensions
IMG_HEIGHT = 256
IMG_WIDTH = 8192  # 4096 * 2
CHANNELS = 1

# Paths to the directories containing images
lhsa_kanjur_path = '/Resized_Selected_LG'
computer_font_path = '/Resized_Selected_Comp'

def load_images_from_folder(folder, img_height, img_width, channels):
    images = []
    for filename in sorted(os.listdir(folder)):
        file_path = os.path.join(folder, filename)
        if os.path.isdir(file_path):
            continue  # Skip directories
        try:
            img = load_img(file_path, color_mode='grayscale', target_size=(img_height, img_width))
            if img is not None:
                img_array = img_to_array(img)
                images.append(img_array)
        except Exception as e:
            print(f"Error loading image {file_path}: {e}")
    images = np.array(images).astype('float32')
    images = (images - 127.5) / 127.5  # Normalize images to [-1, 1]
    return images

# Load images
lhsa_kanjur_images = load_images_from_folder(lhsa_kanjur_path, IMG_HEIGHT, IMG_WIDTH, CHANNELS)
computer_font_images = load_images_from_folder(computer_font_path, IMG_HEIGHT, IMG_WIDTH, CHANNELS)

# Split the data into training and validation sets
train_lhsa_kanjur, val_lhsa_kanjur, train_computer_font, val_computer_font = train_test_split(
    lhsa_kanjur_images, computer_font_images, test_size=0.2, random_state=42)

def create_dataset(input_images, target_images, batch_size=1):
    dataset = tf.data.Dataset.from_tensor_slices((input_images, target_images))
    dataset = dataset.shuffle(buffer_size=len(input_images))
    dataset = dataset.batch(batch_size)
    return dataset

# Create TensorFlow datasets
batch_size = 1
train_dataset = create_dataset(train_computer_font, train_lhsa_kanjur, batch_size)
val_dataset = create_dataset(val_computer_font, val_lhsa_kanjur, batch_size)

2024-07-02 04:31:04.100744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22283 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:82:00.0, compute capability: 8.9


In [4]:
# Residual Block
def residual_block(x, filters, kernel_size=(3, 3)):
    r = Conv2D(filters, kernel_size, padding='same')(x)
    r = BatchNormalization()(r)
    r = ReLU()(r)
    r = Conv2D(filters, kernel_size, padding='same')(r)
    r = BatchNormalization()(r)
    r = Add()([r, x])
    return r

# Generator with reduced features in latent space (2x32)
def build_generator(input_shape):
    inputs = Input(input_shape)  # (256, 8192, 1)
    # Encoder
    e1 = Conv2D(32, (4, 4), strides=2, padding='same')(inputs)  # (128, 4096, 32)
    e2 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(64, (4, 4), strides=2, padding='same')(e1)))  # (64, 2048, 64)
    e3 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(128, (4, 4), strides=2, padding='same')(e2)))  # (32, 1024, 128)
    e4 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(256, (4, 4), strides=2, padding='same')(e3)))  # (16, 512, 256)
    e5 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(512, (4, 4), strides=2, padding='same')(e4)))  # (8, 256, 512)
    e6 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(1024, (4, 4), strides=2, padding='same')(e5)))  # (4, 128, 1024)
    e7 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(2048, (4, 4), strides=2, padding='same')(e6)))  # (2, 64, 2048)
    
    # Adding Residual Blocks to the Latent Space
    r = residual_block(e7, 2048)  # (2, 64, 2048)
    r = residual_block(r, 2048)  # (2, 64, 2048)
    
    # Decoder with additional layers and skip connections
    d1 = ReLU()(BatchNormalization()(Conv2DTranspose(1024, (4, 4), strides=2, padding='same')(r)))  # (4, 128, 1024)
    d1 = Concatenate()([d1, e6])  # (4, 128, 2048)
    d1 = residual_block(d1, 2048)  # (4, 128, 2048)
    
    d2 = ReLU()(BatchNormalization()(Conv2DTranspose(512, (4, 4), strides=2, padding='same')(d1)))  # (8, 256, 512)
    d2 = Concatenate()([d2, e5])  # (8, 256, 1024)
    d2 = residual_block(d2, 1024)  # (8, 256, 1024)
    
    d3 = ReLU()(BatchNormalization()(Conv2DTranspose(256, (4, 4), strides=2, padding='same')(d2)))  # (16, 512, 256)
    d3 = Concatenate()([d3, e4])  # (16, 512, 512)
    d3 = residual_block(d3, 512)  # (16, 512, 512)
    
    d4 = ReLU()(BatchNormalization()(Conv2DTranspose(128, (4, 4), strides=2, padding='same')(d3)))  # (32, 1024, 128)
    d4 = Concatenate()([d4, e3])  # (32, 1024, 256)
    d4 = residual_block(d4, 256)  # (32, 1024, 256)
    
    d5 = ReLU()(BatchNormalization()(Conv2DTranspose(64, (4, 4), strides=2, padding='same')(d4)))  # (64, 2048, 64)
    d5 = Concatenate()([d5, e2])  # (64, 2048, 128)
    d5 = residual_block(d5, 128)  # (64, 2048, 128)

    d6 = ReLU()(BatchNormalization()(Conv2DTranspose(32, (4, 4), strides=2, padding='same')(d5)))  # (128, 4096, 32)
    d6 = Concatenate()([d6, e1])  # (128, 4096, 64)
    d6 = residual_block(d6, 64)  # (128, 4096, 64)
    
    d7 = Conv2DTranspose(1, (4, 4), strides=2, padding='same', activation='tanh')(d6)  # (256, 8192, 1)
    
    return Model(inputs, d7)

In [5]:
# Discriminator with added complexity
def build_discriminator(input_shape):
    inputs = Input(input_shape)  # (256, 8192, 1)
    target = Input(input_shape)  # (256, 8192, 1)
    combined = Concatenate()([inputs, target])  # (256, 8192, 2)
    
    d1 = LeakyReLU(alpha=0.2)(Conv2D(64, (4, 4), strides=2, padding='same')(combined))  # (128, 4096, 64)
    d2 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(128, (4, 4), strides=2, padding='same')(d1)))  # (64, 2048, 128)
    d3 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(256, (4, 4), strides=2, padding='same')(d2)))  # (32, 1024, 256)
    d4 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(512, (4, 4), strides=2, padding='same')(d3)))  # (16, 512, 512)
    d5 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(1024, (4, 4), strides=2, padding='same')(d4)))  # (8, 256, 1024)
    d6 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(2048, (4, 4), strides=2, padding='same')(d5)))  # (4, 128, 2048)
    d7 = LeakyReLU(alpha=0.2)(BatchNormalization()(Conv2D(4096, (4, 4), strides=2, padding='same')(d6)))  # (2, 64, 4096)
    
    d8 = Conv2D(1, (4, 4), strides=1, padding='same')(d7)  # (2, 64, 1)
    
    return Model([inputs, target], d8)

In [6]:
class Pix2Pix(Model):
    def __init__(self, generator, discriminator, lambda_value=100):  # You can adjust lambda_value here
        super(Pix2Pix, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_value = lambda_value
        self.generator_loss_tracker = tf.keras.metrics.Mean(name="generator_loss")
        self.discriminator_loss_tracker = tf.keras.metrics.Mean(name="discriminator_loss")

    def compile(self, generator_optimizer, discriminator_optimizer, gen_loss_fn, disc_loss_fn, **kwargs):
        super(Pix2Pix, self).compile(**kwargs)
        self.generator_optimizer = generator_optimizer
        self.discriminator_optimizer = discriminator_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn

    @property
    def metrics(self):
        return [self.generator_loss_tracker, self.discriminator_loss_tracker]

    def train_step(self, data):
        input_image, target = data

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            fake_image = self.generator(input_image, training=True)

            disc_real_output = self.discriminator([input_image, target], training=True)
            disc_fake_output = self.discriminator([input_image, fake_image], training=True)

            gen_gan_loss = self.gen_loss_fn(tf.ones_like(disc_fake_output), disc_fake_output)
            gen_l1_loss = tf.reduce_mean(tf.abs(target - fake_image))
            gen_total_loss = gen_gan_loss + (self.lambda_value * gen_l1_loss)

            disc_real_loss = self.disc_loss_fn(tf.ones_like(disc_real_output), disc_real_output)
            disc_fake_loss = self.disc_loss_fn(tf.zeros_like(disc_fake_output), disc_fake_output)
            disc_total_loss = disc_real_loss + disc_fake_loss

        generator_gradients = gen_tape.gradient(gen_total_loss, self.generator.trainable_variables)
        discriminator_gradients = disc_tape.gradient(disc_total_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(generator_gradients, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.discriminator.trainable_variables))

        self.generator_loss_tracker.update_state(gen_total_loss)
        self.discriminator_loss_tracker.update_state(disc_total_loss)

        return {"generator_loss": self.generator_loss_tracker.result(), "discriminator_loss": self.discriminator_loss_tracker.result()}

    def test_step(self, data):
        input_image, target = data
        fake_image = self.generator(input_image, training=False)

        disc_real_output = self.discriminator([input_image, target], training=False)
        disc_fake_output = self.discriminator([input_image, fake_image], training=False)

        gen_gan_loss = self.gen_loss_fn(tf.ones_like(disc_fake_output), disc_fake_output)
        gen_l1_loss = tf.reduce_mean(tf.abs(target - fake_image))
        gen_total_loss = gen_gan_loss + (self.lambda_value * gen_l1_loss)

        disc_real_loss = self.disc_loss_fn(tf.ones_like(disc_real_output), disc_real_output)
        disc_fake_loss = self.disc_loss_fn(tf.zeros_like(disc_fake_output), disc_fake_output)
        disc_total_loss = disc_real_loss + disc_fake_loss

        return {"generator_loss": gen_total_loss, "discriminator_loss": disc_total_loss}


In [7]:
# Instantiate the Pix2Pix model
input_shape = (256, 8192, 1)
generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)

# Define optimizers
generator_optimizer = Adam(2e-4, beta_1=0.5)
discriminator_optimizer = Adam(2e-4, beta_1=0.5)

# Instantiate and compile the Pix2Pix model
pix2pix_model = Pix2Pix(generator, discriminator, lambda_value=100)

pix2pix_model.compile(
    generator_optimizer=generator_optimizer,
    discriminator_optimizer=discriminator_optimizer,
    gen_loss_fn=MeanSquaredError(),
    disc_loss_fn=MeanSquaredError()
)

In [8]:
# Set epochs
EPOCHS = 10

# Train the model
history = pix2pix_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

Epoch 1/10


2024-07-02 04:31:45.556951: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 80.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-02 04:31:45.557009: W tensorflow/core/kernels/gpu_utils.cc:54] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2024-07-02 04:31:45.578379: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-07-02 04:31:45.691923: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 88.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if mor

NotFoundError: Graph execution error:

Detected at node model/conv2d_1/Conv2D defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 17, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 529, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipykern

In [ ]:
import matplotlib.pyplot as plt

def generate_and_display_images(model, test_input, target, index):
    prediction = model(test_input, training=False)
    plt.figure(figsize=(15, 45))  # Increase the figure size to ensure high quality and images on separate lines
    
    display_list = [test_input[0], target[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']
    
    for i in range(3):
        plt.subplot(3, 1, i + 1)  # Change the layout to 3 rows, 1 column
        plt.title(title[i])
        plt.imshow(display_list[i] * 0.5 + 0.5, cmap='gray')  # Rescale to [0, 1] for visualization
        plt.axis('off')
    plt.show()

def evaluate_and_visualize(model, val_dataset, num_images=1):  # Set default to 1 to print only one set of images
    for i, (input_image, target) in enumerate(val_dataset.take(num_images)):
        generate_and_display_images(model.generator, input_image, target, i)

# Visualize the results on examples from the validation dataset
evaluate_and_visualize(pix2pix_model, val_dataset, num_images=1)  # Set to 1 to print only one set of images
